In [45]:
import requests
import pandas as pd
from joblib import Parallel, delayed
from tqdm import tqdm

import basedosdados as bd
import pandas as pd 
import plotly.express as px
import numpy as np

pd.set_option('display.max_columns', 500) # ampliar o limite de exibição das colunas para 500


In [2]:
df_dados = pd.read_csv("https://docs.google.com/spreadsheets/d/e/2PACX-1vTQvGH2nk_Mycr6sW2Mt0NEE404AVxtdEWrwNridewLdu9zcP_t0RAOOxihcj4XLobhg0oxuuoLwggg/pub?gid=1811600516&single=true&output=csv",
                      dtype = 'str')

df_dados.head()

,DATA PGTO,ANO,CPF SERVIDOR,CPF/CNPJ FORNECEDOR,NOME FORNECEDOR,VALOR,TIPO,SUBELEMENTO DE DESPESA,CDIC
0,04/01/2019,2019,***.135.521-**,03696869000100,BIG TRANS COMERCIAL DE ALIMENTOS LTDA,"R$ 303,78",C,GENEROS DE ALIMENTA€ÇO,00140.000911/2018-70.R.08.01/01/2019/31/12/2018.N
1,04/01/2019,2019,***.135.521-**,17871449001957,SHOPPING DOS COSMÉTICOS,"R$ 63,70",C,MATERIAL DE LIMPEZA E PROD. DE HIGIENIZA€ÇO,00140.000911/2018-70.R.08.01/01/2019/31/12/2018.N
2,04/01/2019,2019,***.135.521-**,12137325000108,PEIXARIA GUARA LTDA,"R$ 3.749,99",C,GENEROS DE ALIMENTA€ÇO,00140.000911/2018-70.R.08.01/01/2019/31/12/2018.N
3,04/01/2019,2019,***.135.521-**,03696869000100,BIG TRANS COMERCIAL DE ALIMENTOS LTDA,"R$ 42,02",C,GENEROS DE ALIMENTA€ÇO,00140.000911/2018-70.R.08.01/01/2019/31/12/2018.N
4,04/01/2019,2019,***.135.521-**,01610583000135,DISTRIBUIDORA DE CARNE MORRO BRANCO LTDA - ME,"R$ 614,88",C,GENEROS DE ALIMENTA€ÇO,00140.000911/2018-70.R.08.01/01/2019/31/12/2018.N


In [ ]:
df_dados["CPF/CNPJ FORNECEDOR"].describe()

count              13338
unique              1858
top       03696869000100
freq                1495
Name: CPF/CNPJ FORNECEDOR, dtype: object

In [ ]:
# df_dados["CPF/CNPJ FORNECEDOR"] = df_dados["CPF/CNPJ FORNECEDOR"].astype(str)

In [ ]:
df_dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13339 entries, 0 to 13338
Data columns (total 9 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   DATA PGTO               13339 non-null  object
 1   ANO                     13339 non-null  object
 2   CPF SERVIDOR            13339 non-null  object
 3   CPF/CNPJ FORNECEDOR     13338 non-null  object
 4   NOME FORNECEDOR         13339 non-null  object
 5   VALOR                   13339 non-null  object
 6   TIPO                    13339 non-null  object
 7   SUBELEMENTO DE DESPESA  13339 non-null  object
 8   CDIC                    13339 non-null  object
dtypes: object(9)
memory usage: 938.0+ KB


### Coletando dados adicionais dos CNPJs fornecedores

In [ ]:
pais = []
uf = []
municipio = []
bairro = []
logradouro = []
nome_fantasia = []
socios = []

for cnpj in tqdm(df_dados["CPF/CNPJ FORNECEDOR"]):
  
  try:
    response = requests.get(f"https://brasilapi.com.br/api/cnpj/v1/{cnpj}")
    dados_cnpj = response.json()
  except:
    dados_cnpj = []
  
  try:
    pais.append(dados_cnpj["pais"])
  except:
    pais.append("NaN")

  try:
    uf.append(dados_cnpj["uf"])
  except:
    uf.append("NaN")
  
  try:
    municipio.append(dados_cnpj["municipio"])
  except:
    municipio.append("NaN")
  
  try:
    bairro.append(dados_cnpj["bairro"])
  except:
    bairro.append("NaN")

  try:
    logradouro.append(dados_cnpj["logradouro"])
  except:
    logradouro.append("NaN")
  
  try:
    nome_fantasia.append(dados_cnpj["nome_fantasia"])
  except:
    nome_fantasia.append("NaN")

  try:
    socios.append(dados_cnpj["qsa"])
  except:
    socios.append("NaN")

  # print(f'{response}: {cnpj}')


df_dados["pais"] = pais
df_dados["uf"] = uf
df_dados["municipio"] = municipio
df_dados["bairro"] = bairro
df_dados["logradouro"] = logradouro
df_dados["nome_fantasia"] = nome_fantasia
df_dados["socios"] = socios



100%|██████████| 13339/13339 [19:59<00:00, 11.12it/s]


In [ ]:
df_dados.to_csv("cartao_coporativo_bolsonaro_com_dados_cnpj.csv")


### Coletando dados de sócios

In [ ]:
df_teste = df_dados.head(10)

In [ ]:
df_teste

In [5]:
pais = []
uf = []
municipio = []
bairro = []
logradouro = []
nome_fantasia = []
nomes_socios_1 = []
cpf_socios_1 = []
faixa_etaria_socios_1 = []
quant_socios_geral = []


for cnpj in tqdm(df_dados["CPF/CNPJ FORNECEDOR"]):
  
  try:
    response = requests.get(f"https://brasilapi.com.br/api/cnpj/v1/{cnpj}")
    dados_cnpj = response.json()
  except:
    dados_cnpj = []
  
  try:
    pais.append(dados_cnpj["pais"])
  except:
    pais.append("NaN")

  try:
    uf.append(dados_cnpj["uf"])
  except:
    uf.append("NaN")
  
  try:
    municipio.append(dados_cnpj["municipio"])
  except:
    municipio.append("NaN")
  
  try:
    bairro.append(dados_cnpj["bairro"])
  except:
    bairro.append("NaN")

  try:
    logradouro.append(dados_cnpj["logradouro"])
  except:
    logradouro.append("NaN")
  
  try:
    nome_fantasia.append(dados_cnpj["nome_fantasia"])
  except:
    nome_fantasia.append("NaN")

  try:
    nomes_socios = []
    cpf_socios = []
    faixa_etaria_socios = []

    if len(dados_cnpj["qsa"]) < 1:
      quant_socios = 0
    else: 
      quant_socios = len(dados_cnpj["qsa"])

    for item in dados_cnpj["qsa"]:
      nomes_socios.append(item["nome_socio"])
      cpf_socios.append(item["cnpj_cpf_do_socio"])
      faixa_etaria_socios.append(item["faixa_etaria"])

    nomes_socios_1.append(nomes_socios)
    cpf_socios_1.append(cpf_socios)
    faixa_etaria_socios_1.append(faixa_etaria_socios)
    quant_socios_geral.append(quant_socios)
      
  except:
    # socios.append("NaN")
    nomes_socios_1.append("NaN")
    cpf_socios_1.append("NaN")
    faixa_etaria_socios_1.append("NaN")
    quant_socios_geral.append("NaN")

  # print(f'{response}: {cnpj}')


df_dados["pais"] = pais
df_dados["uf"] = uf
df_dados["municipio"] = municipio
df_dados["bairro"] = bairro
df_dados["logradouro"] = logradouro
df_dados["nome_fantasia"] = nome_fantasia
df_dados["quant_socios"] = quant_socios_geral
df_dados["nomes_socios"] = nomes_socios_1
df_dados["cpf_socios"] = cpf_socios_1
df_dados["faixa_etaria_socios"] = faixa_etaria_socios_1




  6%|▌         | 770/13339 [04:26<57:16,  3.66it/s]  

In [ ]:
df_dados["quant_socios"].describe()

count     13339
unique       20
top           2
freq       5366
Name: quant_socios, dtype: int64

In [ ]:
df_dados.to_csv("cartao_coporativo_bolsonaro_com_socios.csv")


### Testatando API de CNPJ

In [ ]:
cnpj = "27865757000102"
response = requests.get(f"https://brasilapi.com.br/api/cnpj/v1/{cnpj}")
print(response)

<Response [200]>


In [ ]:
dados_cnpj = response.json()
dados_cnpj["cnpj"]

'27865757000102'

In [ ]:

nomes_socios = []
cpf_socios = []
faixa_etaria_socios = []

for item in dados_cnpj["qsa"]:
  print(item)
  print(item["nome_socio"])
  nomes_socios.append(item["nome_socio"])
  cpf_socios.append(item["cnpj_cpf_do_socio"])
  faixa_etaria_socios.append(item["faixa_etaria"])
  
  

{'pais': None, 'nome_socio': 'PAULO DAUDT MARINHO', 'codigo_pais': None, 'faixa_etaria': 'Entre 41 a 50 anos', 'cnpj_cpf_do_socio': '***048947**', 'qualificacao_socio': 'Diretor', 'codigo_faixa_etaria': 5, 'data_entrada_sociedade': '2020-01-16', 'identificador_de_socio': 2, 'cpf_representante_legal': '***000000**', 'nome_representante_legal': '', 'codigo_qualificacao_socio': 10, 'qualificacao_representante_legal': 'Não informada', 'codigo_qualificacao_representante_legal': 0}
PAULO DAUDT MARINHO
{'pais': None, 'nome_socio': 'ERICK DE MIRANDA BRETAS', 'codigo_pais': None, 'faixa_etaria': 'Entre 41 a 50 anos', 'cnpj_cpf_do_socio': '***223047**', 'qualificacao_socio': 'Diretor', 'codigo_faixa_etaria': 5, 'data_entrada_sociedade': '2020-01-16', 'identificador_de_socio': 2, 'cpf_representante_legal': '***000000**', 'nome_representante_legal': '', 'codigo_qualificacao_socio': 10, 'qualificacao_representante_legal': 'Não informada', 'codigo_qualificacao_representante_legal': 0}
ERICK DE MIRAN

In [ ]:
len(dados_cnpj["qsa"])

5

In [ ]:
print(nomes_socios)

['PAULO DAUDT MARINHO', 'ERICK DE MIRANDA BRETAS', 'CLAUDIA FALCAO DA MOTTA', 'RAYMUNDO COSTA PINTO BARROS', 'AMAURI SERGIO SOARES']


In [ ]:
response.json()

{'uf': 'RJ',
 'cep': '22460901',
 'qsa': [{'pais': None,
   'nome_socio': 'PAULO DAUDT MARINHO',
   'codigo_pais': None,
   'faixa_etaria': 'Entre 41 a 50 anos',
   'cnpj_cpf_do_socio': '***048947**',
   'qualificacao_socio': 'Diretor',
   'codigo_faixa_etaria': 5,
   'data_entrada_sociedade': '2020-01-16',
   'identificador_de_socio': 2,
   'cpf_representante_legal': '***000000**',
   'nome_representante_legal': '',
   'codigo_qualificacao_socio': 10,
   'qualificacao_representante_legal': 'Não informada',
   'codigo_qualificacao_representante_legal': 0},
  {'pais': None,
   'nome_socio': 'ERICK DE MIRANDA BRETAS',
   'codigo_pais': None,
   'faixa_etaria': 'Entre 41 a 50 anos',
   'cnpj_cpf_do_socio': '***223047**',
   'qualificacao_socio': 'Diretor',
   'codigo_faixa_etaria': 5,
   'data_entrada_sociedade': '2020-01-16',
   'identificador_de_socio': 2,
   'cpf_representante_legal': '***000000**',
   'nome_representante_legal': '',
   'codigo_qualificacao_socio': 10,
   'qualificacao

### Cruzando sócios das empresas fornecedoras com os doadores de campanha

In [3]:
df_socios_prestadores = pd.read_csv("cartao_coporativo_bolsonaro_com_socios.csv")
df_socios_prestadores.head()

,Unnamed: 0,DATA PGTO,ANO,CPF SERVIDOR,CPF/CNPJ FORNECEDOR,NOME FORNECEDOR,VALOR,TIPO,SUBELEMENTO DE DESPESA,CDIC,pais,uf,municipio,bairro,logradouro,nome_fantasia,quant_socios,nomes_socios,cpf_socios,faixa_etaria_socios
0,0,04/01/2019,2019,***.135.521-**,3.696869e+12,BIG TRANS COMERCIAL DE ALIMENTOS LTDA,"R$ 303,78",C,GENEROS DE ALIMENTA€ÇO,00140.000911/2018-70.R.08.01/01/2019/31/12/2018.N,NaN,DF,BRASILIA,ASA NORTE,SHCN EQ 402/403,BIG BOX SUPERMERCADOS,2.0,"['JORGE HELOU FILHO', 'MARIO HABKA']","['***414011**', '***983451**']","['Entre 61 a 70 anos', 'Entre 61 a 70 anos']"
1,1,04/01/2019,2019,***.135.521-**,1.787145e+13,SHOPPING DOS COSMÉTICOS,"R$ 63,70",C,MATERIAL DE LIMPEZA E PROD. DE HIGIENIZA€ÇO,00140.000911/2018-70.R.08.01/01/2019/31/12/2018.N,NaN,DF,BRASILIA,ASA NORTE,SDN (SETOR DE DIVERSOES NORTE),SHOPPING DOS COSMETICOS,2.0,"['ROBSON MACHADO SILVA', 'THAIS MACHADO DOS SA...","['***421871**', '***087231**']","['Entre 51 a 60 anos', 'Entre 21 a 30 anos']"
2,2,04/01/2019,2019,***.135.521-**,1.213733e+13,PEIXARIA GUARA LTDA,"R$ 3.749,99",C,GENEROS DE ALIMENTA€ÇO,00140.000911/2018-70.R.08.01/01/2019/31/12/2018.N,NaN,DF,BRASILIA,ASA SUL,SHCS CL 408,VEDA PESCADOS,1.0,['AYLA MARTA DA SILVA FERNANDES'],['***010223**'],['Entre 21 a 30 anos']
3,3,04/01/2019,2019,***.135.521-**,3.696869e+12,BIG TRANS COMERCIAL DE ALIMENTOS LTDA,"R$ 42,02",C,GENEROS DE ALIMENTA€ÇO,00140.000911/2018-70.R.08.01/01/2019/31/12/2018.N,NaN,DF,BRASILIA,ASA NORTE,SHCN EQ 402/403,BIG BOX SUPERMERCADOS,2.0,"['JORGE HELOU FILHO', 'MARIO HABKA']","['***414011**', '***983451**']","['Entre 61 a 70 anos', 'Entre 61 a 70 anos']"
4,4,04/01/2019,2019,***.135.521-**,1.610583e+12,DISTRIBUIDORA DE CARNE MORRO BRANCO LTDA - ME,"R$ 614,88",C,GENEROS DE ALIMENTA€ÇO,00140.000911/2018-70.R.08.01/01/2019/31/12/2018.N,NaN,DF,BRASILIA,ASA NORTE,LOC SCLN QD 404 BLOCO E,DISTRIBUIDORA DE CARNE MORRO BRANCO,2.0,"['RONALDO CAETANO DE BRITO', 'CARLOS EDUARDO A...","['***621301**', '***044181**']","['Entre 61 a 70 anos', 'Entre 31 a 40 anos']"


In [11]:
consulta_sql = """
SELECT nome_candidato, especie_receita, data_receita, valor_receita, sigla_uf_doador, nome_doador_rf, numero_documento_doacao, cpf_cnpj_doador

FROM `basedosdados.br_tse_eleicoes.receitas_candidato` 

WHERE nome_candidato = "JAIR MESSIAS BOLSONARO" AND ano = 2022

"""

df_consulta = bd.read_sql(consulta_sql, billing_project_id="dadostatu")


Downloading: 100%|██████████| 392164/392164 [01:06<00:00, 5927.12rows/s]


In [13]:
df_consulta.head()


,nome_candidato,especie_receita,data_receita,valor_receita,sigla_uf_doador,nome_doador_rf,numero_documento_doacao,cpf_cnpj_doador
0,JAIR MESSIAS BOLSONARO,pix,2022-10-24,30.00,None,ENAURA SOUZA DO NASCIMENTO,2162931473,21701148587
1,JAIR MESSIAS BOLSONARO,pix,2022-09-28,50.00,None,LUCAS IVAR PRIMO,2025214606,04520293928
2,JAIR MESSIAS BOLSONARO,pix,2022-10-24,1.00,None,SIMIAO GERVANI KRUGER,2168383227,03092554736
3,JAIR MESSIAS BOLSONARO,pix,2022-10-21,108.22,None,SARI MARIANA COSTA GASPAR CORTE REAL,2156082538,08272827450
4,JAIR MESSIAS BOLSONARO,pix,2022-10-31,50.00,None,ANTONIO GIONA,2201646533,15833992915


In [19]:
df_consulta.sort_values("cpf_cnpj_doador")

,nome_candidato,especie_receita,data_receita,valor_receita,sigla_uf_doador,nome_doador_rf,numero_documento_doacao,cpf_cnpj_doador
41368,JAIR MESSIAS BOLSONARO,pix,2022-09-12,1.00,None,IEDA SANTANA SANTOS,1945724236,00000100579
336028,JAIR MESSIAS BOLSONARO,pix,2022-10-05,1.00,None,GILIANA FERREIRA DE JESUS,2063527516,00000216283
4811,JAIR MESSIAS BOLSONARO,pix,2022-10-31,2.00,None,CHESTER MAIK SANCHES,2199366500,00000295213
366231,JAIR MESSIAS BOLSONARO,pix,2022-10-13,10.00,None,RAQUEL RODRIGUES BARBOSA,2117007295,00000330140
162407,JAIR MESSIAS BOLSONARO,pix,2022-09-13,1.00,None,KENNEDY ALEXANDRE BARCELLOS,1953794667,00000341770
...,...,...,...,...,...,...,...,...
352978,JAIR MESSIAS BOLSONARO,pix,2022-10-13,1.00,None,GILDA SOUZA DANTAS,2116944385,99999315534
27750,JAIR MESSIAS BOLSONARO,pix,2022-09-12,1.00,None,TIAGO KROLL,1946182216,99999757049
2315,JAIR MESSIAS BOLSONARO,--,2022-09-30,85485.03,None,None,None,None
2316,JAIR MESSIAS BOLSONARO,--,2022-10-31,29567.32,None,None,None,None


In [34]:
df_doadores_bolsonaro = df_consulta.groupby(["cpf_cnpj_doador", "nome_doador_rf"])["valor_receita"].sum()
df_doadores_bolsonaro.head()

cpf_cnpj_doador  nome_doador_rf             
00000100579      IEDA SANTANA SANTOS             1.0
00000216283      GILIANA FERREIRA DE JESUS       1.0
00000295213      CHESTER MAIK SANCHES            2.0
00000330140      RAQUEL RODRIGUES BARBOSA       10.0
00000341770      KENNEDY ALEXANDRE BARCELLOS     1.0
Name: valor_receita, dtype: float64

In [36]:
df_doadores_bolsonaro = df_doadores_bolsonaro.reset_index()

In [37]:
df_doadores_bolsonaro.head()

,cpf_cnpj_doador,nome_doador_rf,valor_receita
0,00000100579,IEDA SANTANA SANTOS,1.0
1,00000216283,GILIANA FERREIRA DE JESUS,1.0
2,00000295213,CHESTER MAIK SANCHES,2.0
3,00000330140,RAQUEL RODRIGUES BARBOSA,10.0
4,00000341770,KENNEDY ALEXANDRE BARCELLOS,1.0


In [73]:
lista_doadores_prestadores = []

for doador in df_doadores_bolsonaro.values:
    # print(doador[-1])
    nome_doador = doador[1]
    nome_doador = str(nome_doador)
    valor_doador = doador[2]
    cpf_doador = doador[0]
    cpf_doador = str(cpf_doador)

    if cpf_doador.startswith("00000"):
        cpf_doador = cpf_doador[5:]
        # print(cpf_doador)
    else:
        cpf_doador = cpf_doador[3:]
        cpf_doador = cpf_doador[:-2]
        # print(cpf_doador)

    # if df_socios_prestadores["nomes_socios"].str.contains(nome_doador):
    if df_socios_prestadores["nomes_socios"].str.contains(nome_doador).any() and df_socios_prestadores["cpf_socios"].str.contains(cpf_doador).any():
        print(f"Achei: {nome_doador} | CPF: {cpf_doador} | Valor: {valor_doador} ")

        print("-------")
    # print("-------")

Achei: FABIOLA ZANCANARO QUEIROZ GRECCO | CPF: 271081 | Valor: 200.0 
-------
Achei: JUAN ALESSANDRO HERNANDEZ DIBO | CPF: 644271 | Valor: 1.0 
-------
Achei: MARIA DO SOCORRO CAVALCANTE MUGGLER | CPF: 162101 | Valor: 50.0 
-------


KeyboardInterrupt: 

In [71]:
df_socios_prestadores.head(1)

,Unnamed: 0,DATA PGTO,ANO,CPF SERVIDOR,CPF/CNPJ FORNECEDOR,NOME FORNECEDOR,VALOR,TIPO,SUBELEMENTO DE DESPESA,CDIC,pais,uf,municipio,bairro,logradouro,nome_fantasia,quant_socios,nomes_socios,cpf_socios,faixa_etaria_socios
0,0,04/01/2019,2019,***.135.521-**,3.696869e+12,BIG TRANS COMERCIAL DE ALIMENTOS LTDA,"R$ 303,78",C,GENEROS DE ALIMENTA€ÇO,00140.000911/2018-70.R.08.01/01/2019/31/12/2018.N,NaN,DF,BRASILIA,ASA NORTE,SHCN EQ 402/403,BIG BOX SUPERMERCADOS,2.0,"['JORGE HELOU FILHO', 'MARIO HABKA']","['***414011**', '***983451**']","['Entre 61 a 70 anos', 'Entre 61 a 70 anos']"


In [66]:
df_socios_prestadores["nomes_socios"].values

array(["['JORGE HELOU FILHO', 'MARIO HABKA']",
       "['ROBSON MACHADO SILVA', 'THAIS MACHADO DOS SANTOS HONORATO']",
       "['AYLA MARTA DA SILVA FERNANDES']", ...,
       "['GUILLAUME MARIE DIDIER GRAS', 'FREDERIC GARCIA', 'MARCELO RIBEIRO PIMENTEL', 'JOAQUIM ALEXANDRE FERNANDES SOUSA']",
       "['MARCELO LUIZ RAMOS']", "['JORGE HELOU FILHO', 'MARIO HABKA']"],
      dtype=object)